In [3]:
import pickle

with open("./out/ts-train-merges.txt", "rb") as f:
  merges = pickle.load(f)

with open("./out/ts-train-vocab.txt", "rb") as f:
  vocab = pickle.load(f)

print(merges[:10])
print(list(vocab.items())[:10])

longest_vocab_token = sorted(list(vocab.values()), key=lambda x: len(x), reverse=True)[0]
print(longest_vocab_token)

[(b' ', b't'), (b'h', b'e'), (b' ', b'a'), (b' ', b's'), (b' ', b'w'), (b'n', b'd'), (b' t', b'he'), (b'e', b'd'), (b' ', b'b'), (b' t', b'o')]
[(0, b'<|endoftext|>'), (1, b'\x00'), (2, b'\x01'), (3, b'\x02'), (4, b'\x03'), (5, b'\x04'), (6, b'\x05'), (7, b'\x06'), (8, b'\x07'), (9, b'\x08')]
b' accomplishment'


In [ ]:
for merge in merges:
  print(merge)

In [ ]:
for key, token in vocab.items():
  print(token, key)

In [ ]:
# COMPRESSION RATIO

from cs336_basics.tokenizer import Tokenizer

tokenizer = Tokenizer.from_files("./out/ts-train-vocab.txt", "./out/ts-train-merges.txt", ["<|endoftext|>"])

# ids = tokenizer.encode("This is some text.")
# text = tokenizer.decode(ids)

# print(ids)
# print(text)

data_path = "./data/TinyStoriesV2-GPT4-valid.txt"

with open(data_path) as f:
  contents = f.read()

# Sample 10 docs
docs = contents.split("<|endoftext|>")[:10]

ids = []
doc_bytes = []

for doc in docs:
  # print(doc)
  ids.extend(tokenizer.encode(doc))
  doc_bytes.extend(bytes(doc, encoding="UTF-8"))

num_tokens = len(ids)
num_bytes = len(doc_bytes)

print(num_bytes / num_tokens)

4.011164867882397


In [17]:
# TOKENIZE TINYSTORIES AND/OR OWT

from cs336_basics.tokenizer import Tokenizer

tokenizer = Tokenizer.from_files("./out/ts-train-vocab.txt", "./out/ts-train-merges.txt", ["<|endoftext|>"])

data_path = "./data/TinyStoriesV2-GPT4-valid.txt"

ids = []

with open(data_path) as f:
    tokens = tokenizer.encode_iterable(f)
    for i, token in enumerate(tokens):
        print(token)
        ids.append(token)

        if i > 1000:
            break

print(len(ids))

118
862
492
499
266
322
608
370
263
911
465
45
338
1965
2594
349
1603
285
3147
534
391
888
328
263
390
477
47
339
283
378
797
267
263
3147
1081
550
266
2555
336
47
316
4354
4217
336
267
392
1098
474
888
47
285
3147
364
505
342
690
358
47
11
0
11
430
439
259
398
45
313
259
1045
267
1162
954
45
401
283
259
346
3594
47
438
390
496
402
415
528
266
325
810
263
3594
47
527
327
45
415
1041
342
669
475
47
316
283
378
494
47
11
734
541
342
358
45
546
45
266
433
474
2505
387
263
475
47
312
503
899
267
3224
45
408
362
466
365
570
263
475
47
415
324
45
317
74
1128
622
475
1063
713
263
3594
397
11
1338
267
415
427
1378
266
263
3594
47
312
405
608
45
408
362
407
266
570
263
669
475
47
312
503
713
263
3594
45
408
309
283
422
1277
266
538
47
415
324
45
317
1146
1416
480
313
267
2505
387
622
475
397
11
723
427
713
263
3594
266
2505
47
424
283
1277
267
1449
47
312
466
365
570
263
475
47
312
652
266
644
484
45
408
263
3594
283
422
1617
47
415
267
546
405
1434
313
263
3594
47
312
1022
387
433
45
408
544
7